# Mixture of Linear Regressions

In this note, we will solve the mixture of linear regressions problem using the moment method.

This block contains some $\LaTeX$ macros.
$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [1]:
import util
from util import *
import models
from models.LinearRegressions import LinearRegressionsMixture
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
import sympy as sp
sp.init_printing()

Let $w_1, ..., w_K \in \Re^D$ be a set of $K$ regression coefficients. Let $x_1, ..., x_N \in \Re^D$ be a given set of data points and let $y_n = \sum_{k} \delta_k w_k^\top x_n + \epsilon$ be observed responses. Our objective is to recover $(w_k)$.

## Toy Example

Let's construct a toy example for the rest of this document.

In [2]:
K, D = 2, 2
np.random.seed(0)
pis = np.array([0.4, 0.6])
ws = np.array([[0.75, 0.25], [0.4, 0.9]])
model = LinearRegressionsMixture.generate(K, D, betas = ws, weights = pis, cov =  1.0)
print("True parameters:")
print(model.weights)
print(model.betas)
print(model.sigma)

True parameters:
[ 0.4  0.6]
[[ 0.75  0.25]
 [ 0.4   0.9 ]]
[[ 1.  0.]
 [ 0.  1.]]


# Noiseless, Infinite Data Setting

In this scenario, assume that $\epsilon = 0$ and that we observe expected moments of the data. Consider the moments:
\begin{align}
\E[x^\alpha y^b] 
&= \sum_k \pi_k \E[x^\alpha y^b | \delta_k = 1] \\
&= \sum_k \pi_k \E[x^\alpha [w_k^\top x]^b] \\
&= \sum_k \pi_k \E[x^\alpha \sum_{\beta \in p(b)} w_k^\beta x^\beta] \\
&= \sum_{\beta \in p(b)} \E_\pi[w^\beta] \E[x^{\alpha + \beta}],
\end{align}
where $p(b)$ are the $d$-partitions of $b$. Note that $\E[x^{\alpha + \beta}]$ are observable quantities, and thus this simply represents a mixture over the polynomials with terms $w_k^\beta$.

In [3]:
def evaluate_mixture(ws, pis, beta):
    """
    Compute E_\pi[w^beta]
    """
    return sum(pi * util.monomial(w, beta) for w, pi in zip(ws.T, pis))
    
def compute_exact_y_moments(ws, pis, moments_x, alpha, b):
    """
    Compute the exact moments E[x^a y^b] using coefficients ws, pis and moments_x.
    """
    D, K = ws.shape
    ret = 0.
    coeffs = sp.ntheory.multinomial_coefficients(D,b)
    for beta in partitions(D, b):
        ret += coeffs[beta] * evaluate_mixture(ws, pis, beta) * moments_x[tuple_add(alpha, beta)]
    return ret

def describe_moment_polynomial(R, moments_x, moment_y, alpha, b):
    """
    Computes the moment polynomial for E[x^alpha, y^b]
    """
    D = len(R.symbols)
    w = R.symbols
    expr = -moment_y
    coeffs = sp.ntheory.multinomial_coefficients(D,b)
    for beta in partitions(D, b):
        expr += coeffs[beta] * util.monomial(w, beta) * moments_x[tuple_add(alpha, beta)]
    return expr

In [4]:
# Example 
def double_factorial(n): 
    return reduce(mul, xrange(n, 0, -2)) if n > 0 else 1
def gaussian_moments(sigma, d):
    """
    E[x^d] where x is standard gaussian with sigma
    """
    if d == 0: return 1
    elif d % 2 == 0: return double_factorial(d-1) * sigma**d
    else: return 0
def expected_gaussian_moments(sigma, alphas):
    return {alpha : prod(gaussian_moments(sigma, a) for a in alpha) for alpha in alphas}
def expected_uniform_moments(alphas):
    return {alpha : 1. for alpha in alphas}
def expected_moments_y(ws, pis, moments_x, alphabs):
    return {(alpha, b) : compute_exact_y_moments(ws, pis, moments_x, alpha, b) for alpha, b in alphabs}

In [5]:
R, _ = sp.xring(['w%d'%d for d in xrange(D)], sp.RR, sp.grevlex)

deg_b, deg_x = 3, 2
sigma = model.sigma[0,0]
alphas = list(dominated_elements((deg_x for _ in xrange(D))))
alphabs = [(alpha, b) for alpha in alphas for b in xrange(1,deg_b+1)]
alphas_ = list(dominated_elements((deg_x + deg_b for _ in xrange(D))))
moments_x = expected_gaussian_moments(sigma, alphas_)
moments_y = expected_moments_y(ws, pis, moments_x, alphabs)
#display(moments)

In [6]:
display(moments_x)

In [7]:
display(moments_y)

In [8]:
def get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b):
    constrs = []
    for b in xrange(1, deg_b+1):
        for alpha in util.dominated_elements((deg_x for _ in xrange(D))):
            constrs.append( describe_moment_polynomial(R, moments_x, moments_y[(alpha, b)], alpha, b) )
    return constrs

With this machinery, we can compute the moment polynomials required for the moment method magic!

In [9]:
from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)

constrs = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
M = MomentMatrix(3, R.symbols, morder='grevlex')
sol = solvers.solve_generalized_mom_coneqp(M, constrs, 3)
display( model.betas)
display(extractors.extract_solutions_lasserre(M, sol['x'], Kmax=2))

display(extractors.extract_solutions_dreesen_proto(M, sol['x'], Kmax=2))


     pcost       dcost       gap    pres   dres
 0:  3.3331e-01 -1.1934e+00  4e+01  4e+00  8e+00
 1:  1.9518e-01 -3.2675e+00  7e+00  8e-01  1e+00
 2:  1.4298e-01 -1.0911e+00  1e+00  1e-01  2e-01
 3:  3.5067e-02 -1.2531e-01  2e-01  1e-02  2e-02
 4:  5.2528e-03 -1.8015e-02  2e-02  9e-04  2e-03
 5:  8.6614e-04 -2.3142e-03  3e-03  9e-05  2e-04
 6:  2.5471e-04 -8.0417e-05  3e-04  9e-07  2e-06
 7:  5.9055e-05  2.3865e-06  6e-05  1e-07  2e-07
 8:  4.0805e-05  2.3551e-05  2e-05  1e-08  2e-08
 9:  3.0154e-05  2.7546e-05  3e-06  1e-09  2e-09
10:  2.9128e-05  2.8320e-05  8e-07  2e-10  3e-10
11:  2.8647e-05  2.8507e-05  1e-07  2e-11  4e-11
12:  2.8587e-05  2.8549e-05  4e-08  4e-12  7e-12
Optimal solution found.


array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

{w0: array([ 0.25,  0.75]), w1: array([ 0.9,  0.4])}

the next biggest eigenvalue we are losing is 0.000687


In [10]:
display(constrs)

## With real samples

Let's try to solve the problem with generated samples.


In [11]:
def compute_expected_moments(xs, alphas):
    moments = {alpha : 0. for alpha in alphas}
    for alpha in alphas:
        m = monomial(xs.T, alpha)
        moments[alpha] = m if isinstance(m,float) else float(m.mean())
    return moments

def compute_expected_moments_y(ys, xs, alphabs):
    moments = {(alpha, b) : 0. for alpha, b in alphabs}
    for alpha, b in alphabs:
        moments[(alpha,b)] = float((monomial(xs.T, alpha) * ys**b).mean())
    return moments

In [30]:
from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)
from util import fix_parameters, column_rerr
totalerror = 0
for t in range(10):
    xs, ys = model.sample(1e5)
    moments_x = compute_expected_moments(xs, alphas_)
    moments_y = compute_expected_moments_y(ys, xs, alphabs)

    constrs_sdp = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
    M = MomentMatrix(3, R.symbols, morder='grevlex')
    solsdp = solvers.solve_generalized_mom_coneqp(M, constrs_sdp, None)
    #sol = solvers.solve_moments_with_convexiterations(M, constrs, 3)
    display(model.betas)
    betas_lassare = extractors.extract_solutions_lasserre(M, solsdp['x'], Kmax=2)
    betas_lassare_array = np.array([betas_lassare[R.symbols[0]], betas_lassare[R.symbols[1]]])
    betas_lassare_array = closest_permuted_matrix(model.betas, betas_lassare_array)
    display(betas_lassare_array)
    totalerror += column_rerr(betas_lassare_array, model.betas)
print totalerror

     pcost       dcost       gap    pres   dres
 0:  3.3598e-01 -1.1946e+00  4e+01  4e+00  8e+00
 1:  2.0013e-01 -3.2737e+00  7e+00  8e-01  1e+00
 2:  1.4751e-01 -1.0900e+00  1e+00  1e-01  2e-01
 3:  3.8639e-02 -1.2252e-01  2e-01  1e-02  2e-02
 4:  8.6934e-03 -1.4568e-02  2e-02  9e-04  2e-03
 5:  4.3139e-03  1.1255e-03  3e-03  9e-05  2e-04
 6:  3.7003e-03  3.3689e-03  3e-04  7e-07  1e-06
 7:  3.5036e-03  3.4485e-03  6e-05  8e-08  1e-07
 8:  3.4839e-03  3.4707e-03  1e-05  6e-09  1e-08
 9:  3.4759e-03  3.4741e-03  2e-06  7e-10  1e-09
10:  3.4752e-03  3.4748e-03  4e-07  9e-11  2e-10
11:  3.4750e-03  3.4749e-03  6e-08  1e-11  2e-11
Optimal solution found.


array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

array([[ 0.932,  0.397],
       [ 0.276,  0.775]])

     pcost       dcost       gap    pres   dres
 0:  3.3331e-01 -1.1855e+00  4e+01  4e+00  8e+00
 1:  1.9731e-01 -3.2335e+00  7e+00  8e-01  1e+00
 2:  1.4408e-01 -1.0799e+00  1e+00  1e-01  2e-01
 3:  3.6006e-02 -1.2415e-01  2e-01  1e-02  2e-02
 4:  6.2513e-03 -1.7167e-02  2e-02  9e-04  2e-03
 5:  1.8249e-03 -1.4257e-03  3e-03  9e-05  2e-04
 6:  1.1837e-03  8.5090e-04  3e-04  8e-07  1e-06
 7:  9.7987e-04  9.2727e-04  5e-05  8e-08  2e-07
 8:  9.5929e-04  9.4701e-04  1e-05  7e-09  1e-08
 9:  9.5166e-04  9.5010e-04  2e-06  9e-10  2e-09
10:  9.5106e-04  9.5080e-04  3e-07  8e-11  2e-10
11:  9.5093e-04  9.5089e-04  4e-08  1e-11  2e-11
Optimal solution found.


array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

...

array([[ 0.893,  0.408],
       [ 0.295,  0.742]])

     pcost       dcost       gap    pres   dres
 0:  3.3348e-01 -1.1983e+00  4e+01  4e+00  8e+00
 1:  1.9892e-01 -3.2821e+00  7e+00  8e-01  1e+00
 2:  1.4720e-01 -1.0917e+00  1e+00  1e-01  2e-01
 3:  3.8392e-02 -1.2218e-01  2e-01  1e-02  2e-02
 4:  8.1127e-03 -1.5401e-02  2e-02  9e-04  2e-03
 5:  3.6481e-03  4.3902e-04  3e-03  9e-05  2e-04
 6:  3.0375e-03  2.7019e-03  3e-04  9e-07  2e-06
 7:  2.8411e-03  2.7849e-03  6e-05  1e-07  2e-07
 8:  2.8242e-03  2.8055e-03  2e-05  1e-08  3e-08
 9:  2.8131e-03  2.8101e-03  3e-06  2e-09  3e-09
10:  2.8123e-03  2.8111e-03  1e-06  5e-10  9e-10
11:  2.8116e-03  2.8115e-03  2e-07  7e-11  1e-10
12:  2.8116e-03  2.8115e-03  6e-08  2e-11  3e-11
Optimal solution found.


array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

array([[ 0.742,  0.563],
       [ 0.458,  0.59 ]])

2.93365288138
